## Wikipedia text 정제 및 문장분리
##### 20231110

## 1. 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import stanza
# Stanza 초기화
stanza.download('id')  # 영어 모델 다운로드
nlp = stanza.Pipeline('id', processors='tokenize')
from tqdm import tqdm

## 2. 데이터 불러오기

In [ ]:
df = pd.read_csv('C:\\Users\\USER\\Desktop\\wiki수동합치기2.csv')
df

In [ ]:
df = df.drop_duplicates(subset = 'title')
df

## 3. 정규표현식 함수 정의 및 정제

In [6]:
def process_test(text):

    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"<ref[^<]*</ref>", "", text)
    text = re.sub(r"\[\[[^\|\]]*\|", "[[", text)
    text = re.sub(r"{{[^}]*}}", "", text)
    text = re.sub(r"{[^}]*}", "", text)
    text = re.sub(r"<[^>]*>", "", text)
    #text = re.sub(r"\|\d+px", "", text) #remove image px
    text = re.sub(r"==\s*([^=]+)\s*=+", "", text) #소제목 제거
    text = re.sub(r"\=", "", text)
    text = re.sub(r"\[http[^\]]+\]", "", text) #http 제거
    text = re.sub(r"\[\[Kategori:[^\]]+\]\]","", text) #카테고리 제거
    text = re.sub(r"\{", "", text)
    text = re.sub(r"\}", "", text)
    text = re.sub(r"\\'","", text) #wiki에서 글씨체 효과 제거
    text = re.sub(r"\'''", "", text) #wiki에서 bold처리 표현식 제거
    text = re.sub(r"\''", "", text) #wiki에서 이탤릭체 표현식 제거
    text = re.sub(r"\#", "", text) #wiki 숫자 표시 제거
    text = re.sub(r"\*", "", text) #wiki 점 제거
    text = re.sub(r"\[", "", text)
    text = re.sub(r"\]", "", text)
    #text = re.sub(r"\|[^|]+\|", "|", text)
    #text = re.sub(r"\|.*? |\|\d+px", "", text) #remove | and image px
    text = re.sub(r'jmpl\|', '', text)
    text = re.sub(r'ka\|', '', text)
    text = re.sub(r'\d+px\|', '', text)
    text = re.sub(r'kiri\|', '', text)
    text = re.sub(r'thumb\|', '', text)
    text = re.sub(r'lurus\|', '', text)
    text = re.sub(r'left\|', '', text)
    text = re.sub(r'right\|', '', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',"",text)
    text = re.sub(r"\|.*?   |\|\d+px","",text)
    text = re.sub(r"\|.*?  |\|\d+px","",text)
    text = re.sub(r"\|.*? |\|\d+px","",text)
    text = re.sub(r"\|.*?|\|\d+px","",text)
    text = re.sub(r"\d+px", "",text) #remove image px
    text = re.sub(r"[가-힣]+","",text) #remove 한글
    text = re.sub(r"[一-龥]+","",text) #remove 한자
    text = re.sub(r"\([^)]*\)","",text) #remove 소괄호, 소괄호 속 내용
    text = re.sub(r"http\S+", "", text) #remove http
    text = re.sub(r"\!","",text)
    text = re.sub(r'.*?\.jpg\s*', '', text) #jpg 앞쪽 문장 모두 제거
    text = re.sub(r'jpg', '', text)
    text = re.sub(r'Berkas:.*',"",text) #Berkas: 로 문장 시작 시 모두 제거
    text = re.sub(r'\s*section at the bottom of the .*$','',text)
    text = re.sub(r'text = re.sub(r"[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF『』、。《》]", "", text)','',text) #remove 일본어
    text = re.sub(r'-->',"",text)
    text = re.sub(r'-',"",text)
    text = re.sub(r'::',"",text)
    text = re.sub(r'《',"",text)
    text = re.sub(r'》',"",text)
    text = re.sub(r'『',"",text)
    text = re.sub(r'』',"",text)
    text = re.sub(r'&x7C;',"",text)
    

    return text

##### 정규표현식 함수 적용 전 NaN 값 처리

In [ ]:
# NaN 값을 처리합니다. 예를 들어, NaN 값을 빈 문자열로 대체합니다.
df['text'] = df['text'].fillna('')

# 모든 값을 문자열로 변환합니다.
df['text'] = df['text'].astype(str)


In [ ]:
df['title'] = df['title'].fillna('')

# 모든 값을 문자열로 변환합니다.
df['title'] = df['title'].astype(str)

##### 'text' 열에 정규표현식 함수 적용

In [ ]:
df['cleansing_test'] = df['text'].apply(process_test)
df

## 4. 정제된 문장 분리 및 토크나이징

In [ ]:
def tokenizing(df):
    max_length=5
    new_rows = []
    index=0
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
        index+=1
        article_text = row['cleansing_test']
        doc = nlp(article_text)
        sen_id = 1  # 문장 ID를 1로 초기화
        for sentence in doc.sentences:
            new_row = row.copy()
            new_row['Sentence'] = sentence.text
            new_row['Token']= [token.text for token in sentence.tokens]
            new_row['Word_Count'] = len(new_row['Token'])
            new_row['Doc_ID']='20230816_wikidata_'+new_row['title']+'_000001'
            new_row['Sen_ID'] = new_row['Doc_ID']+'_sen'+str('{:06d}'.format(sen_id))
            new_row['Pub_Date']=new_row['date'][:10]
            new_row['Title']=new_row['title']
            new_row['Pub_Subj']=new_row['title']
            new_rows.append(new_row)
            sen_id += 1  # 문장 ID 증가

    new_df = pd.DataFrame(new_rows)
    new_df['Tokenized_Sentence']= new_df['Token'].apply(' '.join)
    new_df['Col_Date']='2023-08-10'
    new_df['Pub_Type']='Wikipedia'
    new_df['Text']=new_df['cleansing_test']
    new_df['Filename']='C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NIA AI 데이터셋 구축 - General/06-수집데이터/위키 데이터/rawdata/idwiki-20230720-pages-articles-multistream.xml'

    new_df.drop_duplicates(subset='Sentence', inplace=True)
    new_df.reset_index(drop=True, inplace=True)
    new_df = new_df[['Doc_ID', 'Filename', 'Title', 'Pub_Type', 'Pub_Subj', 'Pub_Date', 'Col_Date', 'Sen_ID', 'Word_Count', 'Text', 'Sentence','Tokenized_Sentence' , 'Token']]
    new_df = new_df[new_df['Token'].apply(lambda x: len(x) > max_length)]
    return new_df

In [ ]:
df_test= tokenizing(df)
df_test

In [ ]:
df_test['Word_Count'].sum() #총 토큰 수 확인

2235631

##### 테스트웍스(가공 담당 회사)전달 전 각 Token에 대한 column 생성

In [ ]:
df = df_test[df_test['Word_Count']<=200]
df

In [ ]:
import ast  # 문자열을 리스트로 변환하는 데 사용

# 빈 데이터프레임 리스트 초기화
temp_dfs = []

# Token 열의 문자열을 실제 리스트로 변환
df['Token'] = df['Token'].apply(ast.literal_eval)

# 가장 많은 토큰을 가진 행의 토큰 개수를 찾음
max_tokens = df['Token'].apply(len).max()

# 필요한 만큼의 새 열을 미리 생성
for i in range(max_tokens):
    df[f'Column_{i}'] = None

# 각 행에 대해 반복
for idx in tqdm(df.index):
    # 해당 행의 토큰 리스트
    tokens = df.loc[idx, 'Token']

    # 각 토큰에 대해 반복
    for i, token in enumerate(tokens):
        # 해당 토큰을 적절한 열에 할당
        df.at[idx, f'Column_{i}'] = token

In [ ]:
#위 코드 에러 발생 시 이 코드로 컬럼 생성

# 토큰별 작업용 컬럼 생성
for i in range(df['Word_Count'].max()):
    new_column_name = f'Column_{i}'  # 새 열 이름 생성
    df[new_column_name]=np.nan

for idx in tqdm(df.index):
    for i, token in enumerate(df['Token'][idx]):
        new_column_name = f'Column_{i}'  # 새 열 이름 생성
        df[new_column_name][idx]=token

In [ ]:
# 'Sen_ID'기준으로 정렬
df.sort_values('Sen_ID', inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [55]:
df.to_csv('20231110_원천데이터(위키).csv', index=False, encoding='utf-8-sig')

In [ ]:
df.to_pickle('20231110_원천데이터(위키).pickle') #용량 클 시 pickle 파일로 저장